In [5]:
%pip install psycopg2-binary
%pip install pandas
%pip install sqlalchemy


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 39.3 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [greenlet]
   ---------------------------------------- 0/2 [greenlet]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlal

In [13]:
import pandas as pd
from sqlalchemy import create_engine

# --- AACT PostgreSQL Database Credentials ---
USERNAME = "rexr238"
PASSWORD = "RexR%409113407564"
HOST = "aact-db.ctti-clinicaltrials.org"
PORT = "5432"
DATABASE = "aact"

# --- SQLAlchemy connection string ---
DATABASE_URL = f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"

# --- SQL Query to fetch 1000 studies ---
SQL_QUERY = """
SELECT DISTINCT ON (s.nct_id)
    s.nct_id,
    s.phase,
    c.name AS condition,
    i.intervention_type,
    sp.agency_class AS sponsor_type,
    s.enrollment,
    s.overall_status AS status,
    o.title AS outcome_label
FROM studies s
LEFT JOIN conditions c ON s.nct_id = c.nct_id
LEFT JOIN interventions i ON s.nct_id = i.nct_id
LEFT JOIN sponsors sp ON s.nct_id = sp.nct_id
LEFT JOIN outcomes o ON s.nct_id = o.nct_id
LIMIT 1000;


"""

def fetch_trials():
    try:
        # Create connection engine
        engine = create_engine(DATABASE_URL)

        # Fetch data
        df = pd.read_sql_query(SQL_QUERY, engine)
        print(df.head())  # Print first few rows

        # Optionally save to CSV
        df.to_csv("aact_trials_sample.csv", index=False)
        print("Saved to aact_trials_sample.csv")

    except Exception as e:
        print("Error occurred:", e)

# Run
if __name__ == "__main__":
    fetch_trials()


        nct_id          phase                       condition  \
0  NCT00000102  PHASE1/PHASE2  Congenital Adrenal Hyperplasia   
1  NCT00000104           None                  Lead Poisoning   
2  NCT00000105           None                          Cancer   
3  NCT00000106             NA              Rheumatic Diseases   
4  NCT00000107           None       Heart Defects, Congenital   

  intervention_type sponsor_type  enrollment      status outcome_label  
0              DRUG          NIH         NaN   COMPLETED          None  
1         PROCEDURE          FED         NaN   COMPLETED          None  
2        BIOLOGICAL        OTHER       112.0  TERMINATED          None  
3            DEVICE          NIH         NaN     UNKNOWN          None  
4              None          NIH         NaN   COMPLETED          None  
Saved to aact_trials_sample.csv


In [15]:
df = pd.read_csv("aact_trials_sample.csv")  # or from SQL query directly
print(df.head())
print(df.columns)


        nct_id          phase                       condition  \
0  NCT00000102  PHASE1/PHASE2  Congenital Adrenal Hyperplasia   
1  NCT00000104            NaN                  Lead Poisoning   
2  NCT00000105            NaN                          Cancer   
3  NCT00000106            NaN              Rheumatic Diseases   
4  NCT00000107            NaN       Heart Defects, Congenital   

  intervention_type sponsor_type  enrollment      status outcome_label  
0              DRUG          NIH         NaN   COMPLETED           NaN  
1         PROCEDURE          FED         NaN   COMPLETED           NaN  
2        BIOLOGICAL        OTHER       112.0  TERMINATED           NaN  
3            DEVICE          NIH         NaN     UNKNOWN           NaN  
4               NaN          NIH         NaN   COMPLETED           NaN  
Index(['nct_id', 'phase', 'condition', 'intervention_type', 'sponsor_type',
       'enrollment', 'status', 'outcome_label'],
      dtype='object')


In [16]:
print(df.isnull().sum())


nct_id                 0
phase                170
condition              0
intervention_type     47
sponsor_type           0
enrollment           239
status                 0
outcome_label        988
dtype: int64


In [17]:
# See % of missing values per column
(df.isnull().mean() * 100).round(1).sort_values(ascending=False)


outcome_label        98.8
enrollment           23.9
phase                17.0
intervention_type     4.7
condition             0.0
nct_id                0.0
sponsor_type          0.0
status                0.0
dtype: float64

In [18]:
print("Unique values per column:")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique")


Unique values per column:
nct_id: 1000 unique
phase: 6 unique
condition: 134 unique
intervention_type: 7 unique
sponsor_type: 4 unique
enrollment: 232 unique
status: 4 unique
outcome_label: 11 unique


In [19]:
# For small columns like phase, intervention_type, etc.
print("Phases:", df['phase'].dropna().unique())
print("Intervention Types:", df['intervention_type'].dropna().unique())
print("Sponsor Types:", df['sponsor_type'].dropna().unique())
print("Statuses:", df['status'].dropna().unique())


Phases: ['PHASE1/PHASE2' 'PHASE1' 'PHASE3' 'PHASE2' 'PHASE2/PHASE3' 'PHASE4']
Intervention Types: ['DRUG' 'PROCEDURE' 'BIOLOGICAL' 'DEVICE' 'BEHAVIORAL' 'OTHER'
 'DIETARY_SUPPLEMENT']
Sponsor Types: ['NIH' 'FED' 'OTHER' 'INDUSTRY']
Statuses: ['COMPLETED' 'TERMINATED' 'UNKNOWN' 'WITHDRAWN']
